# Downscale & clip large raster data files
- useful if really they can't be downloaded programmatically for the AOI (area of interest)
- decreases storage and RAM req
- raster is often at very high spatial res (eg. 10s of meter for S2), which is not needed for PyPSA

In [ ]:
# running the jupyter notebook on the compute nodes doesnt build the path as expected, you have to manually do this
import os

import IPython

working_directory = os.path.dirname(
    IPython.extract_module_locals()[1]["__vsc_ipynb_file__"]
)
os.chdir(os.path.join(os.path.dirname(working_directory), "scripts"))
os.getcwd()

In [ ]:
import geopandas as gpd
import rioxarray
import xarray as xr
from constants import CRS

## functions

In [ ]:
def xarr_to_gdf(
    xarr: xr.DataArray, var_name: str, x_var="x", y_var="y", crs=CRS
) -> gpd.GeoDataFrame:
    """Convert an xarray to GDF

    Args:
        xarr (xr.DataArray): the input array
        var_name (str): the array variable to be converted.
        x_var (str, optional): the x dimension. Defaults to "x".
        y_var (str, optional): the y dimension. Defaults to "y".
        crs (_type_, optional): the crs. Defaults to CRS.

    Returns:
        gpd.GeoDataFrame: geodata frame in chosen CRS
    """
    df = xarr.to_dataframe()
    df.reset_index(inplace=True)
    return gpd.GeoDataFrame(
        df[var_name], geometry=gpd.points_from_xy(df[x_var], df[y_var]), crs=crs
    )

In [ ]:
def coarsen_and_clip(
    paths: list[os.PathLike],
    region_gdf: gpd.GeoDataFrame,
    region_name: str,
    downscale=10,
):
    """Coarsen high res satellite data and clip to region of interest and Save new files

    Args:
        paths (list[os.PathLike]): the raster paths list
        region_gdf (gpd.GeoDataFrame): the shape file to clip to
        region_name (str): the name of the region (for file identification)
        downscale (int, optional): the factor to corasen by. Defaults to 10.
    """
    for p in paths:
        ds = rioxarray.open_rasterio(p, chunks={"x": 1000, "y": 1000})
        ds = ds.coarsen(x=downscale, y=downscale, boundary="pad").mean()
        ds = ds.rio.clip(region_gdf.geometry.values, region_gdf.crs)
        ds = ds.where(ds <= 100, other=100)
        ds.rio.to_raster(f"{os.path.splitext(p)[0]}_clipped_{region_name}.tif")

# get shape to which to clip

In [ ]:
geo_path = os.path.realpath("../../resources/data/regions/regions_onshore.geojson")
region_shape = gpd.read_file(geo_path)
region_shape.plot()
shape_name = "china"

## downscale data
GLC sentinel analysis is at 100m, downscale to 1x1km2

In [ ]:
paths = [
    os.path.join("../../resources/data/landuse_availability/", raster)
    for raster in ["Bare.tif", "Build_up.tif", "Grass.tif", "Shrubland.tif"]
]
coarsen_and_clip(paths, region_shape, shape_name, downscale=10)

In [ ]:
provs = gpd.read_file("../../resources/data/province_shapes/CHN_adm1.shp")

provs = provs[provs.province.isin(["Anhui", "Jiangsu", "Shanghai"])]
provs_ = gpd.GeoSeries(provs.union_all())

In [ ]:
import shutil

paths_clipped = [
    os.path.join("../../resources/data/landuse_availability/", raster)
    for raster in [
        "Bare_clipped_china.tif",
        "Build_up_clipped_china.tif",
        "Grass_clipped_china.tif",
        "Shrubland_clipped_china.tif",
    ]
]
coarsen_and_clip(paths_clipped, provs_, "tests", 5)
destination_folder = os.path.realpath("../../tests/testdata")
os.makedirs(destination_folder, exist_ok=True)

test_files = []
for p in paths_clipped:
    base_name = os.path.basename(p)
    new_name = os.path.basename(paths_clipped[0]).split("_clipped")[0] + ".tif"
    shutil.move(
        f"{os.path.splitext(p)[0]}_clipped_tests.tif",
        os.path.join(destination_folder, new_name),
    )
    test_files.append(os.path.join(destination_folder, new_name))

In [ ]:
import matplotlib.pyplot as plt

for p in test_files:
    fig, ax = plt.subplots()
    ds = rioxarray.open_rasterio(p)
    ds.plot(ax=ax)

# gepbco bathimetry

In [ ]:
ds = rioxarray.open_rasterio(
    "../../resources/data/landuse_availability/GEBCO_tiff/gebco_2024_CN.tif"
)
# its wg84
ds.rio.write_crs("epsg:4326", inplace=True)
ds.coords["crs"] = 4326
ds

In [ ]:
ds.plot()

In [ ]:
downscale = 24
ds_ = ds.coarsen(x=downscale, y=downscale, boundary="pad").mean()
ds_.rio.write_crs(4326, inplace=True)
# ds_ = ds_.rio.clip(region_shape.geometry.values, region_shape.crs)

In [ ]:
ds_.plot()
ds_.rio.to_raster(
    "../../resources/data/landuse_availability/GEBCO_tiff/gebco_2024_CN_tests.tif"
)